In [1]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [2]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [3]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(f"{continuous_path}/cts_{min_len}/dataset {min_len}_{i}.csv"):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

csv file is loaded from ./data/3_continuous/cts_100/dataset 100_1.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_2.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_3.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_4.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_5.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_6.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_7.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_8.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_9.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_10.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_11.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_12.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_13.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_14.csv
csv file is loa

# future = 10

## Seq2Seq layers

In [4]:
target_list = cts_list

# history size and future size
history_size = 40
future_size = 10
step = 1
                
# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# data to series
history_series = []
future_series = []
for i in range(len(target_list)):
    history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                step, start_idx=0, end_idx=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)

# supervised attention factor
delta = 1.5
att_type = 'exp'
factor = rnn.super_attention(delta, future_size, future_num, att_type)
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers_list = [1, 2]
num_neurons_list = [20, 40, 60]
dense_layers = 1
dense_neurons = 40
model_list = ['seq2seq_gru', 'att_seq2seq_gru', 'datt_seq2seq_gru']

iteration_list = [x for x in range(1,7)]
for iteration in iteration_list:
    for num_layers in num_layers_list:
        for num_neurons in num_neurons_list:
            for model_type in model_list:
                print(f"{iteration}th iteration")
                print(f"num_layers: {num_layers}")
                print(f"num_neurons: {num_neurons}")
                print(f"model_type: {model_type}")
                
                # Dual-attention Seq2Seq model
                RNN_model = rnn.RNN(history_series, history_var, future_series, future_var)
                # TT split
                RNN_model.train_test(test_size=test_size, test_num=test_num)
                # TV split
                valid_size = RNN_model.history_test.shape[0]/RNN_model.history_train.shape[0]
                RNN_model.train_valid(valid_size=valid_size)
                # scaling
                RNN_model.scaling()
                # modeling
                RNN_model.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
                # training
                model_num = iteration
                model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{att_type}_{delta}_{model_num}"
                if not exists(f"./model/{model_name}.h5"):
                    RNN_model.train()
                    RNN_model.save_model(f"./model/{model_name}", 'weights')
                    
                else:
                    RNN_model.model.load_weights(f"./model/{model_name}.h5")
                # test
                test_result = RNN_model.test(False)
                if not exists(f'./result/{model_name}.csv', ):
                    savefile(test_result, './result', model_name)
                print("\n")
print('end opitmization')

1th iteration
num_layers: 1
num_neurons: 20
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 20
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 20
model_type: datt_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 40
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 40
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 40
model_type: datt_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 60
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 60
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 60
model_type: datt_seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 20
model_type: seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 20
model_type: att_seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 20
model_type: datt_seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 40
model_type: seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 40
model_type

## Dense layers

In [5]:
target_list = cts_list

# history size and future size
history_size = 40
future_size = 10
step = 1
                
# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# data to series
history_series = []
future_series = []
for i in range(len(target_list)):
    history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                step, start_idx=0, end_idx=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)

# supervised attention factor
delta = 1.5
att_type = 'exp'
factor = rnn.super_attention(delta, future_size, future_num, att_type)
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 20
dense_layers_list = [1, 2]
dense_neurons_list = [20, 40, 60]
model_list = ['seq2seq_gru', 'att_seq2seq_gru', 'datt_seq2seq_gru']

iteration_list = [x for x in range(1,7)]
for iteration in iteration_list:
    for dense_layers in dense_layers_list:
        for dense_neurons in dense_neurons_list:
            for model_type in model_list:
                print(f"{iteration}th iteration")
                print(f"num_layers: {dense_layers}")
                print(f"num_neurons: {dense_neurons}")
                print(f"model_type: {model_type}")
                
                # Dual-attention Seq2Seq model
                RNN_model = rnn.RNN(history_series, history_var, future_series, future_var)
                # TT split
                RNN_model.train_test(test_size=test_size, test_num=test_num)
                # TV split
                valid_size = RNN_model.history_test.shape[0]/RNN_model.history_train.shape[0]
                RNN_model.train_valid(valid_size=valid_size)
                # scaling
                RNN_model.scaling()
                # modeling
                RNN_model.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
                # training
                model_num = iteration
                model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{att_type}_{delta}_{model_num}"
                if not exists(f"./model/{model_name}.h5"):
                    RNN_model.train()
                    RNN_model.save_model(f"./model/{model_name}", 'weights')
                    
                else:
                    RNN_model.model.load_weights(f"./model/{model_name}.h5")
                # test
                test_result = RNN_model.test(False)
                if not exists(f'./result/{model_name}.csv', ):
                    savefile(test_result, './result', model_name)
                print("\n")
print('end opitmization')

1th iteration
num_layers: 1
num_neurons: 20
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 20
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 20
model_type: datt_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 40
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 40
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 40
model_type: datt_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 60
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 60
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 60
model_type: datt_seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 20
model_type: seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 20
model_type: att_seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 20
model_type: datt_seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 40
model_type: seq2seq_gru


1th iteration
num_layers: 2
num_neurons: 40
model_type

# future = 20

## Seq2Seq layers

In [9]:
target_list = cts_list

# history size and future size
history_size = 40
future_size = 20
step = 1
                
# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# data to series
history_series = []
future_series = []
for i in range(len(target_list)):
    history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                step, start_idx=0, end_idx=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)

# supervised attention factor
delta = 0.5
att_type = 'linear'
factor = rnn.super_attention(delta, future_size, future_num, att_type)
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers_list = [1]
num_neurons_list = [20, 40, 60, 80, 100, 150, 200, 250, 300, 350, 400]
dense_layers = 1
dense_neurons = 40
model_list = ['seq2seq_gru', 'att_seq2seq_gru', 'datt_seq2seq_gru']

iteration_list = [x for x in range(1,7)]
for iteration in iteration_list:
    for num_layers in num_layers_list:
        for num_neurons in num_neurons_list:
            for model_type in model_list:
                print(f"{iteration}th iteration")
                print(f"num_layers: {num_layers}")
                print(f"num_neurons: {num_neurons}")
                print(f"model_type: {model_type}")
                
                # Dual-attention Seq2Seq model
                RNN_model = rnn.RNN(history_series, history_var, future_series, future_var)
                # TT split
                RNN_model.train_test(test_size=test_size, test_num=test_num)
                # TV split
                valid_size = RNN_model.history_test.shape[0]/RNN_model.history_train.shape[0]
                RNN_model.train_valid(valid_size=valid_size)
                # scaling
                RNN_model.scaling()
                # modeling
                RNN_model.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
                # training
                model_num = iteration
                model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{att_type}_{delta}_{model_num}"
                if not exists(f"./model/{model_name}.h5"):
                    RNN_model.train()
                    RNN_model.save_model(f"./model/{model_name}", 'weights')
                    
                else:
                    RNN_model.model.load_weights(f"./model/{model_name}.h5")
                # test
                test_result = RNN_model.test(False)
                if not exists(f'./result/{model_name}.csv', ):
                    savefile(test_result, './result', model_name)
                print("\n")
print('end opitmization')

1th iteration
num_layers: 1
num_neurons: 20
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 20
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 20
model_type: datt_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 40
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 40
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 40
model_type: datt_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 60
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 60
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 60
model_type: datt_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 80
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 80
model_type: att_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 80
model_type: datt_seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 100
model_type: seq2seq_gru


1th iteration
num_layers: 1
num_neurons: 100
model_ty

KeyboardInterrupt: 

## Dense layers

In [4]:
target_list = cts_list

# history size and future size
history_size = 40
future_size = 20
step = 1
                
# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# data to series
history_series = []
future_series = []
for i in range(len(target_list)):
    history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                step, start_idx=0, end_idx=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)

# supervised attention factor
delta = 0.5
att_type = 'linear'
factor = rnn.super_attention(delta, future_size, future_num, att_type)
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 100
dense_layers_list = [1, 2]
dense_neurons_list = [20, 40, 60]
model_list = ['seq2seq_gru', 'att_seq2seq_gru', 'datt_seq2seq_gru']

iteration_list = [x for x in range(1,6)]
for iteration in iteration_list:
    for dense_layers in dense_layers_list:
        for dense_neurons in dense_neurons_list:
            for model_type in model_list:
                print(f"{iteration}th iteration")
                print(f"num_layers: {dense_layers}")
                print(f"num_neurons: {dense_neurons}")
                print(f"model_type: {model_type}")
                
                # Dual-attention Seq2Seq model
                RNN_model = rnn.RNN(history_series, history_var, future_series, future_var)
                # TT split
                RNN_model.train_test(test_size=test_size, test_num=test_num)
                # TV split
                valid_size = RNN_model.history_test.shape[0]/RNN_model.history_train.shape[0]
                RNN_model.train_valid(valid_size=valid_size)
                # scaling
                RNN_model.scaling()
                # modeling
                RNN_model.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
                # training
                model_num = iteration
                model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{att_type}_{delta}_{model_num}"
                if not exists(f"./model/{model_name}.h5"):
                    RNN_model.train()
                    RNN_model.save_model(f"./model/{model_name}", 'weights')
                    
                else:
                    RNN_model.model.load_weights(f"./model/{model_name}.h5")
                # test
                test_result = RNN_model.test(False)
                if not exists(f'./result/{model_name}.csv', ):
                    savefile(test_result, './result', model_name)
                print("\n")
print('end opitmization')

1th iteration
num_layers: 1
num_neurons: 20
model_type: seq2seq_gru
Epoch 1/10000
563/563 - 10s - loss: 0.1323 - val_loss: 0.1230 - 10s/epoch - 19ms/step
Epoch 2/10000
563/563 - 4s - loss: 0.1092 - val_loss: 0.1133 - 4s/epoch - 8ms/step
Epoch 3/10000
563/563 - 4s - loss: 0.1002 - val_loss: 0.1054 - 4s/epoch - 8ms/step
Epoch 4/10000
563/563 - 4s - loss: 0.0926 - val_loss: 0.0958 - 4s/epoch - 8ms/step
Epoch 5/10000
563/563 - 4s - loss: 0.0853 - val_loss: 0.0944 - 4s/epoch - 8ms/step
Epoch 6/10000
563/563 - 4s - loss: 0.0784 - val_loss: 0.0824 - 4s/epoch - 8ms/step
Epoch 7/10000
563/563 - 4s - loss: 0.0722 - val_loss: 0.0795 - 4s/epoch - 8ms/step
Epoch 8/10000
563/563 - 4s - loss: 0.0683 - val_loss: 0.0759 - 4s/epoch - 8ms/step
Epoch 9/10000
563/563 - 4s - loss: 0.0632 - val_loss: 0.0675 - 4s/epoch - 8ms/step
Epoch 10/10000
563/563 - 4s - loss: 0.0601 - val_loss: 0.0660 - 4s/epoch - 8ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.0546 - val_loss: 0.0633 - 5s/epoch - 8ms/step
Epoch 12/10000